# Benchmark models

In [1]:

# Imports
## Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os
from functools import partial

## Views 3
import views_runs
from viewser.operations import fetch
from views_forecasts.extensions import *
from viewser import Queryset, Column



In [2]:

# Common parameters:

dev_id = 'Fatalities002'
run_id = 'Fatalities002'
EndOfHistory = 508
get_future = False

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,396),"predict":(397,456)}
test_partitioner_dict = {"train":(121,444),"predict":(457,504)}
future_partitioner_dict = {"train":(121,492),"predict":(505,512)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/Prediction competition 2023/'


print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)

# Benchmark model parameters
filepath = Mydropbox + 'Prediction_competition_2023/'

year_list = [2018, 2019, 2020, 2021]
draws_cm = 1000
draws_pgm = 100

steps = [3,4,5,6,7,8,9,10,11,12,13,14]
stepcols = ['ln_ged_sb_dep']
for step in steps:
    stepcols.append('step_pred_' + str(step))
print(stepcols)



Dropbox path set to /Users/havardhegre1/Dropbox (ViEWS)/ViEWS/
Overleaf path set to /Users/havardhegre1/Dropbox (ViEWS)/Apps/Overleaf/Prediction competition 2023/
['ln_ged_sb_dep', 'step_pred_3', 'step_pred_4', 'step_pred_5', 'step_pred_6', 'step_pred_7', 'step_pred_8', 'step_pred_9', 'step_pred_10', 'step_pred_11', 'step_pred_12', 'step_pred_13', 'step_pred_14']


In [3]:
from BenchmarkModels import poisson_expand_single_point_predictions, describe_expanded
from BenchmarkModels import poisson_expand_multiple_point_predictions, save_models, save_actuals

# cm models
## Ensemble model predictions

In [4]:
# Expand ensemble predictions:
ensemble_df = pd.DataFrame.forecasts.read_store('cm_ensemble_genetic_test', run=dev_id)[stepcols]
ensemble_df.head()


sc_predictions_ensemble =  poisson_expand_single_point_predictions(ensemble_df=ensemble_df,level='cm',year_list=year_list,draws=1000)

describe_expanded(df=sc_predictions_ensemble[0]['prediction_df'], df_expanded=sc_predictions_ensemble[0]['expanded_df'], month=457, country=57)   
sc_predictions_ensemble[0]['expanded_df'].head()

pr_46_cm_ensemble_genetic_test.parquet


/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

2018
2019
2020
2021
Mean and std of original predictions, all rows:
        prediction
count  2292.000000
mean     46.288810
std     334.649615
min       0.000000
25%       0.017667
50%       0.037432
75%       0.190760
max    6384.809995
Mean and std of expanded predictions, all rows:
        outcome
count   2292000
unique     4176
top           0
freq    1763927
Mean and std of original predictions, one cm:
count     2.00000
unique    2.00000
top       1.05692
freq      1.00000
Name: (457, 57), dtype: float64
Mean and std of expanded predictions, one cm:
        outcome
count      1000
unique        7
top           1
freq        374
Variance: outcome    1.069469
dtype: float64


outcome
month_id country_id draw        
457      1          0          0
                    1          0
                    2          0
                    3          0
                    4          0

## Constituent model predictions

In [5]:
# Retrieve the model predictions
# Using code from the VIEWS fatalities002 system

# 20 models: 1 "draw" from each of 20 constituent models, expanded with 50 draws from Poisson distribution for each model.

from ModelDefinitions import DefineEnsembleModels

level = 'cm'
ModelList_cm = DefineEnsembleModels(level)
ModelList_cm = ModelList_cm[0:20] # Drop Markov models
#ModelList_cm = ModelList_cm[0:3] # For debugging

i = 0
for model in ModelList_cm:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

# Retrieving the predictions for calibration and test partitions
# The ModelList contains the predictions organized by model
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated

ModelList_cm = RetrieveStoredPredictions(ModelList_cm, steps, EndOfHistory, dev_id, level, get_future)

ModelList_cm = CalibratePredictions(ModelList_cm, EndOfHistory, steps)

0 fatalities002_baseline_rf baseline002
1 fatalities002_conflicthistory_rf conflict_ln
2 fatalities002_conflicthistory_gbm conflict_ln
3 fatalities002_conflicthistory_hurdle_lgb conflict_ln
4 fatalities002_conflicthistory_long_xgb conflictlong_ln
5 fatalities002_vdem_hurdle_xgb vdem_short
6 fatalities002_wdi_rf wdi_short
7 fatalities002_topics_rf topics_002
8 fatalities002_topics_xgb topics_002
9 fatalities002_topics_hurdle_lgb topics_002
10 fatalities002_joint_broad_rf joint_broad
11 fatalities002_joint_broad_hurdle_rf joint_broad
12 fatalities002_joint_narrow_xgb joint_narrow
13 fatalities002_joint_narrow_hurdle_xgb joint_narrow
14 fatalities002_joint_narrow_hurdle_lgb joint_narrow
15 fatalities002_all_pca3_xgb all_features
16 fatalities002_aquastat_rf aquastat
17 fatalities002_faostat_rf faostat
18 fatalities002_faoprices_rf faoprices
19 fatalities002_imfweo_rf imfweo
0 fatalities002_baseline_rf
pr_46_cm_fatalities002_baseline_rf_calib.parquet
pr_46_cm_fatalities002_baseline_rf_test

In [6]:
sc_predictions_constituent =  poisson_expand_multiple_point_predictions(ModelList=ModelList_cm,level='cm',year_list=year_list,draws=1000)


fatalities002_baseline_rf


/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

2018
2019
2020
2021
fatalities002_conflicthistory_rf


/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

2018
2019
2020
2021
fatalities002_conflicthistory_gbm


...

2018
2019
2020
2021
fatalities002_faoprices_rf


/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

2018
2019
2020
2021
fatalities002_imfweo_rf


/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

2018
2019
2020
2021
2018
fatalities002_baseline_rf
fatalities002_conflicthistory_rf
fatalities002_conflicthistory_gbm
fatalities002_conflicthistory_hurdle_lgb
fatalities002_conflicthistory_long_xgb
fatalities002_vdem_hurdle_xgb
fatalities002_wdi_rf
fatalities002_topics_rf
fatalities002_topics_xgb
fatalities002_topics_hurdle_lgb
fatalities002_joint_broad_rf
fatalities002_joint_broad_hurdle_rf
fatalities002_joint_narrow_xgb
fatalities002_joint_narrow_hurdle_xgb
fatalities002_joint_narrow_hurdle_lgb
fatalities002_all_pca3_xgb
fatalities002_aquastat_rf
fatalities002_faostat_rf
fatalities002_faoprices_rf
fatalities002_imfweo_rf
2019
fatalities002_baseline_rf
fatalities002_conflicthistory_rf
fatalities002_conflicthistory_gbm
fatalities002_conflicthistory_hurdle_lgb
fatalities002_conflicthistory_long_xgb
fatalities002_vdem_hurdle_xgb
fatalities002_wdi_rf
fatalities002_topics_rf
fatalities002_topics_xgb
fatalities002_topics_hurdle_lgb
fatalities002_joint_broad_rf
fatalities002_joint_broad_hurd

In [7]:
sc_predictions_constituent[0]['expanded_df'].tail()

outcome
month_id country_id draw        
468      246        45         7
                    46        13
                    47        18
                    48        11
                    49        19

# Saving the cm benchmark models

In [8]:
model_names = ['ensemble','constituent']
model_list = [sc_predictions_ensemble,sc_predictions_constituent]

save_models('cm',model_names,model_list, filepath)
save_actuals('cm',ensemble_df, filepath, year_list)

2018
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_ensemble_expanded_2018.parquet
2019
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_ensemble_expanded_2019.parquet
2020
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_ensemble_expanded_2020.parquet
2021
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_ensemble_expanded_2021.parquet
2018
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_constituent_expanded_2018.parquet
2019
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_constituent_expanded_2019.parquet
2020
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_constituent_expanded_2020.parquet
2021
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_cm_constituent_expanded_2021.parquet
                     ged_sb
month_id country_id        
457      1              0.0
        

# pgm level

In [9]:
# Expand ensemble predictions:
ensemble_df = pd.DataFrame.forecasts.read_store('pgm_ensemble_cm_calib_test', run=dev_id)[stepcols]
ensemble_df.head()


sc_predictions_ensemble_pgm = poisson_expand_single_point_predictions(ensemble_df=ensemble_df,year_list=year_list,draws=100,level='pgm')

sc_predictions_ensemble_pgm[0]['expanded_df'].head()



pr_46_pgm_ensemble_cm_calib_test.parquet


/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['prediction'] = np.expm1(df_temp[this_col].values)
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'].loc[this_month] = df_temp['prediction'].values
/Users/havardhegre1/views3/prediction_competition_2023/BenchmarkModels.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

2018
2019
2020
2021


outcome
month_id priogrid_id draw        
457      62356       0          0
                     1          0
                     2          0
                     3          0
                     4          0

In [10]:
# Saving 
model_names = ['ensemble','constituent']
model_list = [sc_predictions_ensemble_pgm]

save_models('pgm',model_names,model_list, filepath)
#save_cm_actuals(ensemble_df, filepath, year_list)




2018
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_pgm_ensemble_expanded_2018.parquet
2019
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_pgm_ensemble_expanded_2019.parquet
2020
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_pgm_ensemble_expanded_2020.parquet
2021
/Users/havardhegre1/Dropbox (ViEWS)/ViEWS/Prediction_competition_2023/bm_pgm_ensemble_expanded_2021.parquet
